Before starting....

In [ ]:
import pandas as pd
import numpy as np 

# store CSV into HDF for performance reasons
files = [
    {
        "filename": "bookings.csv", 
        "fields" : ['arr_port', 'dep_port', 'pax'],
        "mapping" : {'arr_port':np.str, 'dep_port':np.str, 'pax':np.float64}
    },
    {
        "filename": "searches.csv", 
        "fields" : ['Origin', 'Destination'], # means all
        "mapping" : {
            'Origin':np.str,
            'Destination':np.str
        }
    }
]

def toHDF(filedata):
    current_key_id = 1
    filename_hdf = "%s.%s" % (filedata["filename"].split(".")[0], "h5")
    print "Processing %s" % filedata["filename"]
    for dataframe_chunk in pd.read_csv(filedata["filename"], sep="^", chunksize=50000, dtype=filedata["mapping"]):
        #trim column names
        dataframe_chunk.columns = map(lambda x : x.strip(), dataframe_chunk.columns)
        if len(filedata["fields"]) > 0:
            dataframe_chunk[filedata["fields"]].to_hdf(filename_hdf, 'data', table=True, append=True)
        else:
            dataframe_chunk.to_hdf(filename_hdf, current_key, table=True, append=True)
            
        current_key_id += 1

    print "Done..."

for f in files:
    toHDF(f)

Exercise 1

In [ ]:
import pandas as pd
from multiprocessing import Pool
import time 

# ex 1
for f in files:
    store = pd.HDFStore(f["filename"].split(".")[0] + '.h5')
    print "%s has %d lines" % (f["filename"].split(".")[0], store.get_storer('data').nrows)